<a href="https://colab.research.google.com/github/oldaandozerskaya/covid_news/blob/main/aaai_covid_fake_BERT_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import libs

In [1]:
import tensorflow as tf
import pickle, os
import math

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

!pip install pytorch-pretrained-bert pytorch-nlp

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import keras
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from pytorch_pretrained_bert import BertTokenizer, BertConfig, OpenAIGPTModel, OpenAIGPTTokenizer
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

Found GPU at: /device:GPU:0
     |████████████████████████████████| 133kB 4.5MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 6.7MB 11.0MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 1.1MB 7.6MB/s 
     |████████████████████████████████| 3.0MB 14.7MB/s 
     |████████████████████████████████| 1.1MB 51.6MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-cp36-none-any.whl size=91190 sha256=14cb2f7e64f779b813c4357c456ced9f240b5bbec83c1362dcefba18de63494d
  Stored in directory: /root/.cache/pip/wheels/4b/8f/23/7f93e91176acc7c2d651b54fa9f01e3624a47904145c4d133e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2c7ca1c8b5a08ce065a7fb9489b30151f78

#Import data

In [3]:
df = pd.read_csv('/content/drive/My Drive/covid_fake_news/Constraint_English_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [4]:
df['label'].value_counts()

real    3360
fake    3060
Name: label, dtype: int64

In [5]:
train_texts = df.tweet.values
df['label'] = df['label'].map({'fake': 1, 'real': 0})
train_labels = df.label.values

In [6]:
df = pd.read_csv('/content/drive/My Drive/covid_fake_news/Constraint_English_Val.csv')
df.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [7]:
df['label'].value_counts()

real    1120
fake    1020
Name: label, dtype: int64

In [8]:
val_texts = df.tweet.values
df['label'] = df['label'].map({'fake': 1, 'real': 0})
val_labels = df.label.values

#BERT binary classifier

In [9]:
train_sentences = ["[CLS] " + text + " [SEP]" for text in train_texts]
val_sentences = ["[CLS] " + text + " [SEP]" for text in val_texts]

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(text) for text in train_sentences]

print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 890791.99B/s]


Tokenize the first sentence:
['[CLS]', 'the', 'cdc', 'currently', 'reports', '99', '##0', '##31', 'deaths', '.', 'in', 'general', 'the', 'disc', '##re', '##pan', '##cies', 'in', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'ex', '##pl', '##ica', '##ble', '.', 'the', 'death', 'toll', 'stands', 'at', 'roughly', '1000', '##00', 'people', 'today', '.', '[SEP]']


In [11]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=128, dtype="long", truncating="post", padding="post")
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1885 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1112 > 512). Running this sequence through BERT will result in indexing errors


In [12]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train_labels, 
                                                            random_state=42, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=42, test_size=0.2)

In [13]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [14]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [15]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

100%|██████████| 407873900/407873900 [00:11<00:00, 36422517.20B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [16]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [17]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [18]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train loss: 0.1943941493279292


Epoch:  33%|███▎      | 1/3 [03:32<07:04, 212.40s/it]

Validation Accuracy: 0.9658385093167702
Train loss: 0.04940920286508882


Epoch:  67%|██████▋   | 2/3 [07:04<03:32, 212.29s/it]

Validation Accuracy: 0.9642857142857143
Train loss: 0.014407573796933748


Epoch: 100%|██████████| 3/3 [10:35<00:00, 211.86s/it]

Validation Accuracy: 0.9635093167701864


In [20]:
tokenized_texts_val = [tokenizer.tokenize(text) for text in val_sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_val]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=128, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(val_labels)
  
batch_size = 8


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (609 > 512). Running this sequence through BERT will result in indexing errors


In [21]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [22]:
from sklearn.metrics import f1_score, recall_score, precision_score
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]
print('F1-score: %f' % (f1_score(flat_predictions, flat_true_labels)*100))
print('Precision: %f' % (precision_score(flat_predictions, flat_true_labels)*100))
print('Recall: %f' % (recall_score(flat_predictions, flat_true_labels)*100))

F1-score: 97.222222
Precision: 96.078431
Recall: 98.393574


In [25]:
output_dir = '/content/drive/My Drive/covid_fake_news/bert/bert.bin'
torch.save(model.state_dict(), output_dir)

In [26]:
for i, text in enumerate(val_texts):
  if flat_predictions[i]!=flat_true_labels[i]:
    print(flat_predictions[i])
    print(flat_true_labels[i])
    print(text)

0
1
11 out of 13 people (from the Diamond Princess Cruise ship) who had intially tested negative in tests in Japan were later confirmed to be positive in the United States.
0
1
Mike Pence introduces program to cure coronavirus carriers with conversion therapy https://t.co/A36KAO2NWa https://t.co/bp0SDO25F0
0
1
WHO reports record daily increase in global coronavirus cases up over 292000 https://t.co/b0EtsTmqgb
0
1
There is no evidence that children have died because of a COVID-19 vaccine. No vaccine currently in development has been approved for widespread public use. https://t.co/9ecvMR8SAf
0
1
No one should have to pay for coronavirus treatment. Period. https://t.co/GBchrVZmDz
1
0
Gargled water samples may be viable alternative to swabs for detection of COVID-19: Study https://t.co/feOauXK2sh
0
1
The Canadian government has not enacted a law that criminalizes COVID-19 restrictions or prevents enforcement of those guidelines. https://t.co/c7IST1A1ip
0
1
IACC is to Autism as "X" is to C

#Get sentences embeddings

In [11]:
model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

sentence_embeddings = model.encode(train_texts)
with open('/content/drive/My Drive/covid_fake_news/sentence_embeddings_train.pickle', 'wb') as f:
  pickle.dump(sentence_embeddings, f)
sentence_embeddings = model.encode(val_texts)
with open('/content/drive/My Drive/covid_fake_news/sentence_embeddings_val.pickle', 'wb') as f:
  pickle.dump(sentence_embeddings, f)

#Feedforward classifier

In [24]:
with open('/content/drive/My Drive/covid_fake_news/sentence_embeddings_train.pickle', 'rb') as f:
  sentence_embeddings_train = pickle.load(f)

with open('/content/drive/My Drive/covid_fake_news/sentence_embeddings_val.pickle', 'rb') as f:
  sentence_embeddings_val = pickle.load(f)

In [20]:
border = math.ceil(0.8 * sentence_embeddings_train.shape[0])

x_train, x_val = sentence_embeddings_train[:border], sentence_embeddings_train[border:]
y_train, y_val = train_labels[:border], train_labels[border:]

In [40]:
inputs=Input(shape=(1024,), name='input')
x=Dense(1024, activation='tanh', name='fully_connected_1024_tanh')(inputs)
predictions=Dense(2, activation='softmax', name='output_softmax')(x)
model=Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1024)]            0         
_________________________________________________________________
fully_connected_1024_tanh (D (None, 1024)              1049600   
_________________________________________________________________
output_softmax (Dense)       (None, 2)                 2050      
Total params: 1,051,650
Trainable params: 1,051,650
Non-trainable params: 0
_________________________________________________________________


In [46]:
history = model.fit(x_train, y_train, epochs=1, verbose=0, validation_data=(x_val, y_val))
predict = np.argmax(model.predict(sentence_embeddins_test), axis=1)
answer = np.argmax(keras.utils.to_categorical(val_labels,2), axis=1)

f1=f1_score(predict, answer)*100
prec=precision_score(predict, answer)*100
recall=recall_score(predict, answer)*100

print(f1)
print(prec)
print(recall)

90.23773394031362
87.45098039215686
93.20794148380355
